In [1]:
# the following code allows you to import function from
# jupyter notebook in other directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb

In [3]:
class ChiangOGWInvariant():
    def __init__(self, use_recursion_a=True, ogw_1_0_g2=var('x'), debug=False):
        self.use_recursion_a = use_recursion_a
        self.ogw_1_0_g2 = ogw_1_0_g2
        self.debug = debug

    @functools.lru_cache(maxsize=None)
    def chiang_ogw_invariant(self, n, b, k, A, layer=0):
        if isinstance(A, list):
            A = ClassList.compress(A, n)

        l = len(A)      
        if A.invalid_class_list:
            return 0    

        # wall-crossing theorem (Elad)
        if l > 0 and A[0] == -1:
            return -self.chiang_ogw_invariant(n, b, k+1, A[:-1]) 

        #degree axiom (Elad)
        if 2*sum(A) != (n - 3 + 2*b + k + 2*l - k*n):  
            # The maslov index is twice the chern class n+1 for holomorphic spheres. 
            # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
            # p 595 mcduff 
            #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
            if layer == 1:
                print("degree axiom fails")
            return 0    

        # other
        if k == 0 and is_even(b):
            return 0    

        # zero axiom
        if b == 0:
            if k == 1 and l == 1 and A[0] == 0:
                return -1
            else:
                return 0    

        # unit axiom (May)
        # handle all beta = 0 cases
        if l > 0 and A[0] == 0:
            if b == 0 and k == 1 and l == 1:
                return -1
            else:
                return 0    

        # divisor axiom 
        if l > 0 and A[0] == 1:
            #A.pop(0)
            return (b/4)*self.chiang_ogw_invariant(n, b, k, A[1:])   

        # base cases
        if b == 1 and k == 1 and A == []:
            return 3    

        if b == 1 and k == 0 and A == [2]:
            return self.ogw_1_0_g2       

        # recursion (a)
        if l >= 2 and not (k >= 1 and not self.use_recursion_a):
            cor = self.chiang_ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
            col = 0
            ool = 0
            oor = 0
            A_0 = A[0]
            A_1 = A[1]  

            for t in A[2:].bipart():
                I1 = t[0]
                I2 = t[1]
                bipart_coeff = ClassList.bipart_coeff(A[2:], I1)    

                for i in range(0, n+1):
            
                    L1_cor = [1, A_1, i] + I1
                    L2_cor = [n-i, A_0-1] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        cor += bipart_coeff \
                            * (-1)^(b_*(n+1)/2) *  gw_invariant(n, b_, L1_cor) \
                            *  self.chiang_ogw_invariant(n, b1, k, L2_cor)   

                for i in range(0, n+1):
                
                    L1_col = [1, A[0] - 1, i] + I1 
                    L2_col = [n-i, A[1]] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        col += bipart_coeff \
                            * (-1)^((n+1)*b_/2)* gw_invariant(n, b_, L1_col) \
                            * self.chiang_ogw_invariant(n, b1, k, L2_col)    

                L1_ool = [1, A[0]-1] + I1
                L2_ool = [A[1]] + I2
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1
                        ool += bipart_coeff \
                            * binomial(k, k1)*self.chiang_ogw_invariant(n, b1, k1, L1_ool) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_ool)   

                L1_oor = [1, A[1]] + I1
                L2_oor = [A[0] - 1] + I2    
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1 

                        oor += bipart_coeff \
                            * binomial(k, k1) * self.chiang_ogw_invariant(n, b1, k1, L1_oor) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)   

            result = cor - col - oor + ool
            if self.debug:
                print(f"recursion a: {result}")
            return result   

        # recursion (c)
        if l >= 1 and k >= 1:
            co = 0
            ool = 0
            oor = 0
            A_0 = A[0]  

            for t in A.bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A, I1)    

                for i in range(0, n+1):
                    for b1 in range(0, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue

                        co += bipart_coeff \
                            * gw_invariant(n, b_, [A_0 - 1, 1, i] + I1) \
                            * self.chiang_ogw_invariant(n, b, k, [n - i] + I2)   

                for b1 in range(0, b+1):
                    b2 = b - b1 

                    for k1 in range(0, k-1):
                        k2 = k - 1 - k1
                        ool += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1, [A_0 - 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 2, I2)   

                for b1 in range(0, b+1):
                    b2 = b - b2 

                    for k in range(0, k-1):
                        k2 = k - 1 - k1
                        oor += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1 + 1, [A_0 - 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, [1] + I2) 

                result =  -co + ool - oor
                if self.debug:
                    print(f"recursion c : {result}")
                return result   

        # recursion (d)
        # note, when k >= 2 and l == 0,
        # then b >= 2
        if k >= 2 and l == 0:
            sum1 = 0
            sum2 = 0
            sum3 = 0    

            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 4 == 0:
                    b_ = (b - b1) / 4
                else:
                    continue

                for i in range(0, n):
                    sum1 += \
                        gw_invariant(n, b_, [1, 2, i])*self.chiang_ogw_invariant(n, b1, k-1, [n-i, 1]) \
                                + gw_invariant(n, b_, [1, 1, 2, i])*self.chiang_ogw_invariant(n, b1, k-1, [n-i]) 

            for b1 in range(2, b):
                b2 = b + 1 - b1
                for k1 in range(1, k-2):
                    k2 = k - 1 - k1
                    sum2 += binomial(k - 1, k1)*(\
                        self.chiang_ogw_invariant(n, b1, k1, [1, 1])*self.chiang_ogw_invariant(n, b2, k2+1, [2]\
                            - self.chiang_ogw_invariant(n, b1, k1, [1, 2])*self.chiang_ogw_invariant(n, b2, k2+1, [1])))  

            for b1 in range(2, b):
                b2 = b + 1 - b1
                for k1 in range(1, k-2):
                    k2 = k - 2 - k1
                    sum3 += binomial(k - 2, k1)*(\
                        self.chiang_ogw_invariant(n, b1, k1+1, [1, 2])*self.chiang_ogw_invariant(n, b2, k2+1, [1])\
                            + self.chiang_ogw_invariant(n, b1, k1+1, [1])*self.chiang_ogw_invariant(n, b2, k2+1, [1, 2])\
                                - self.chiang_ogw_invariant(n, b1, k2+2, [2])*self.chiang_ogw_invariant(n, b1, k1, [1, 1])\
                                    - self.chiang_ogw_invariant(n, b2, k2+2, [])*self.chiang_ogw_invariant(n, b1, k1, [1, 1, 2])) 

            result = (sum1 + sum2 + sum3)/((b/16 + 1/16)*self.ogw_1_0_g2)
            if self.debug:
                print(f"recursion d: {result}")
            return result   

        print(f"unexpected situation is , b: {b}, k: {k}, A:{A.c_list}, l: {l}")
        return 0

In [4]:
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + 2*b + k + 2*l2 - k*n - 2*l2 * A2
    if (m % (2*(A1 - 1))) == 0:
        return int(m / (2*(A1 - 1)))
    else:
        -1


In [5]:
chiang_ogw = ChiangOGWInvariant(debug=True)
for b in range(0, 6):
    for l2 in [1, 3, 5]:
        A1 = 2
        A2 = 3
        l1 = get_l1(3, b, 0, l2, A1, A2)
        if l1 < 0 :
            continue
        A = [A1]*l1 + [A2]*l2
        chiang = chiang_ogw.chiang_ogw_invariant(3, b, 0, ClassList.compress(A, 3))
        print(f"b: {b}, chiang ogw: {chiang}")

b: 2, chiang ogw: 0
recursion a: 0
b: 3, chiang ogw: 0
b: 4, chiang ogw: 0
recursion c : 0
recursion a: 1/4*x
recursion d: 0
recursion a: 0
recursion c : 0
recursion a: 0
recursion a: 3/8*x
b: 5, chiang ogw: 3/8*x


In [6]:
chiang_ogw_a = ChiangOGWInvariant()
chiang_ogw_c = ChiangOGWInvariant(use_recursion_a=False)

for b in range(1, 5):
    for k in range(1, 7):
        for l2 in range(0, 6):
            A1 = 2
            A2 = 3
        
            l1 = get_l1(3, b, k, l2, A1, A2)
            l1 = 1
            if l1 < 0 :
                continue
            A = [A1]*l1 + [A2]*l2
            chiang_a = chiang_ogw_a.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            chiang_c = chiang_ogw_c.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))

            print(f"b:{b}, k:{k}, A:{A}, chiang_a: {chiang_a}, chiang_c: {chiang_c}")


b:1, k:1, A:[2], chiang_a: 0, chiang_c: 0
b:1, k:1, A:[2, 3], chiang_a: 0, chiang_c: 0
b:1, k:1, A:[2, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:1, A:[2, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:1, A:[2, 3, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:1, A:[2, 3, 3, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:2, A:[2], chiang_a: 0, chiang_c: 0
b:1, k:2, A:[2, 3], chiang_a: 0, chiang_c: 0
b:1, k:2, A:[2, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:2, A:[2, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:2, A:[2, 3, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:2, A:[2, 3, 3, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:3, A:[2], chiang_a: 0, chiang_c: 0
b:1, k:3, A:[2, 3], chiang_a: 0, chiang_c: 0
b:1, k:3, A:[2, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:3, A:[2, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:3, A:[2, 3, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:3, A:[2, 3, 3, 3, 3, 3], chiang_a: 0, chiang_c: 0
b:1, k:4, A:[2], chiang_a: 0, chiang_c: 0
b:1, k:4, A:[2, 3], chiang_a: 0, chiang_c: 0
b:1, k:4, A:[2, 3, 3],